# SageMakerでカスタムコンテナを使った学習
sagemaker-training-toolkitを使ってカスタムコンテナを作り、それを用いてSageMaker上で学習を実施する。

## イメージのビルド
`sagemaker-studio-image-build`をインストールし、`sm-docker` コマンドを使ってイメージをビルドする。

In [1]:
# sagemaker-trainingが含まれているDockerfile
%cat Dockerfile

FROM python:3.9.13-bullseye

RUN pip install -U pip && \
    pip install sagemaker-training sklearn pandas

COPY train.py /opt/ml/code/train.py

ENV SAGEMAKER_PROGRAM train.py


In [2]:
# trainingに使うスクリプト
%pycat train.py

import argparse
import os

import joblib
import pandas as pd
from sklearn import tree

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--max_leaf_nodes", type=int, default=-1)
    args = parser.parse_args()
    channel_train = os.environ["SM_CHANNEL_TRAIN"]  # channelをtrainにしておく必要がある
    model_dir = os.environ["SM_MODEL_DIR"]

    # /opt/ml/input/data/train　以下にあるデータを取得
    print(channel_train)
    input_files = [
        os.path.join(channel_train, file) for file in os.listdir(channel_train)
    ]
    if len(input_files) == 0:
        raise ValueError(f"There are no files in {args.train}.\n")
    raw_data = [pd.read_csv(file, header=None, engine="python") for file in input_files]
    train_data = pd.concat(raw_data)

    # ラベルは最初のカラム
    train_y = train_data.iloc[:, 0]
    train_X = train_data.iloc[:, 1:]

    # scikit-learnのdecision　treeにハイパーパラメータを渡して学習
    max_leaf_nodes = args.max_leaf_nodes
    clf = tree.DecisionTreeClassifier(max_leaf_no

In [3]:
!pip install -U pip
!pip install sagemaker-studio-image-build
!sm-docker build .

...........[Container] 2022/08/26 20:57:20 Waiting for agent ping

[Container] 2022/08/26 20:57:21 Waiting for DOWNLOAD_SOURCE
[Container] 2022/08/26 20:57:24 Phase is DOWNLOAD_SOURCE
[Container] 2022/08/26 20:57:24 CODEBUILD_SRC_DIR=/codebuild/output/src573399070/src
[Container] 2022/08/26 20:57:24 YAML location is /codebuild/output/src573399070/src/buildspec.yml
[Container] 2022/08/26 20:57:24 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2022/08/26 20:57:24 Processing environment variables
[Container] 2022/08/26 20:57:24 No runtime version selected in buildspec.
[Container] 2022/08/26 20:57:24 Moving to directory /codebuild/output/src573399070/src
[Container] 2022/08/26 20:57:24 Configuring ssm agent with target id: codebuild:37f8ae1c-b488-4938-85a9-87730ab6a1cf
[Container] 2022/08/26 20:57:24 Successfully updated ssm agent configuration
[Container] 2022/08/26 20:57:24 Registering with agent
[Container] 2022/08/26 20:57:24 Phases found in YAML: 3
[Container

## トレーニングの実行
image_uriでECRにプッシュしたイメージをしてする以外は、SageMakerで通常のトレーニングを行うときと同様の実装となる。

In [4]:
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()

client = boto3.client("sagemaker-runtime")

instance_count = 1
instance_type = "ml.m4.xlarge"
train_input_s3_path = (
    "s3://sagemaker-us-east-1-980831117329/workshop/sklearn-endpoint/data"
)
train_output_s3_path = (
    "s3://sagemaker-us-east-1-980831117329/workshop/custom-container/output"
)
role = sagemaker.get_execution_role()

image_uri = "980831117329.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio-d-2fqm8ncc55ww:default-1643235874544"

In [5]:
sm = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    role=role,
    instance_count=instance_count,
    instance_type=instance_type,
    max_run=86400,
    output_path=train_output_s3_path,
    sagemaker_session=sagemaker_session,
    hyperparameters={"max_leaf_nodes": 30},
)

sm.fit({"train": train_input_s3_path})

2022-08-26 21:01:30 Starting - Starting the training job...ProfilerReport-1661547690: InProgress
...
2022-08-26 21:02:10 Starting - Preparing the instances for training.........
2022-08-26 21:03:54 Downloading - Downloading input data...
2022-08-26 21:04:25 Training - Training image download completed. Training in progress.
2022-08-26 21:04:25 Uploading - Uploading generated training model2022-08-26 21:04:17,333 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-26 21:04:17,356 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-26 21:04:17,369 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-26 21:04:17,380 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "current_instance_group": "homogen